# Broadcasting


Le *broadcasting* est un mécanisme utilisé par PyTorch pour traiter les tenseurs lors d'opérations arithmétiques non évidentes.

Par exemple, il est clair qu'on ne peut pas additionner une matrice $3 \times 3$ avec une matrice $4 \times 2$, ce qui entraînerait une erreur. En revanche, ajouter un scalaire à une matrice $3 \times 3$ ou un vecteur de taille $3$ à une matrice $3 \times 3$ est possible, même si la logique n'est pas toujours évidente.

Le [broadcasting](https://pytorch.org/docs/stable/notes/broadcasting.html) de PyTorch repose sur des règles simples à connaître lors de la manipulation de tenseurs.


## Règles de broadcasting


Pour que deux tenseurs soient *broadcastables*, ils doivent respecter les règles suivantes :
- Chaque tenseur doit avoir au moins une dimension.
- En itérant sur les tailles des dimensions (en commençant par la dernière), les tailles doivent être égales, ou l'une d'elles doit être 1, ou l'une d'elles doit être absente.

Utilisons des exemples pour clarifier :


In [2]:
import torch

# Deux tenseurs de la même taille sont toujours broadcastables
x=torch.zeros(5,7,3)
y=torch.zeros(5,7,3)

# Les deux tenseurs suivants ne sont pas broadcastables car x n'a pas au moins une dimension
x=torch.zeros((0,))
y=torch.zeros(2,2)

# On aligne les dimensions visuellement pour voir si les tenseurs sont broadcastables
# En partant de la droite,
# 1. x et y ont la même taille et sont de taille 1
# 2. y est de taille 1
# 3. x et y ont la même taille
# 4. la dimension de y n'existe pas
# Les deux tenseurs sont donc broadcastables
x=torch.zeros(5,3,4,1)
y=torch.zeros(  3,1,1)

# A l'inverse, ces deux tenseurs ne sont pas broadcastables car 3. x et y n'ont pas la même taille
x=torch.zeros(5,2,4,1)
y=torch.zeros(  3,1,1)

Maintenant que nous savons comment identifier deux tenseurs *broadcastables*, définissons les règles appliquées lors de l'opération entre eux.

Les règles sont :
- **Règle 1** : Si le nombre de dimensions de x et y diffère, ajoutez 1 au début des dimensions du tenseur ayant le moins de dimensions pour les aligner.
- **Règle 2** : Pour chaque taille de dimension, la taille résultante est le maximum des tailles de x et y.

Le tenseur dont la taille est modifiée sera dupliqué autant que nécessaire pour correspondre.

**Note** : Si deux tenseurs ne sont pas *broadcastables*, leur addition entraînera une erreur. Cependant, dans de nombreux cas, le *broadcasting* fonctionnera mais ne produira pas le résultat souhaité. C'est pourquoi il est crucial de maîtriser ces règles.

Reprenons nos deux exemples :

Ajouter un scalaire à une matrice $3 \times 3$ :


In [12]:
x=torch.randn(3,3)
y=torch.tensor(1)
print("x : " ,x)
print("y : " ,y)
print("x+y : " ,x+y)
print("x+y shape : ",(x+y).shape)
# Le tenseur y est broadcasté pour avoir la même taille que x, il se transforme en tenseur de 1 de taille 3x3

x :  tensor([[ 0.6092, -0.6887,  0.3060],
        [ 1.3496,  1.7739, -0.4011],
        [-0.8876,  0.7196, -0.3810]])
y :  tensor(1)
x+y :  tensor([[1.6092, 0.3113, 1.3060],
        [2.3496, 2.7739, 0.5989],
        [0.1124, 1.7196, 0.6190]])
x+y shape :  torch.Size([3, 3])


Ajouter un vecteur de taille $3$ à une matrice $3 \times 3$ :


In [23]:
x=torch.randn(3,3)
y=torch.tensor([1,2,3]) # tenseur de taille 3
print("x : " ,x)
print("y : " ,y)
print("x+y : " ,x+y)
print("x+y shape : ",(x+y).shape)
# Le tenseur y est broadcasté pour avoir la même taille que x, il se transforme en tenseur de 1 de taille 3x3

x :  tensor([[ 0.9929, -0.1435,  1.5740],
        [ 1.2143,  1.3366,  0.6415],
        [-0.2718,  0.3497, -0.2650]])
y :  tensor([1, 2, 3])
x+y :  tensor([[1.9929, 1.8565, 4.5740],
        [2.2143, 3.3366, 3.6415],
        [0.7282, 2.3497, 2.7350]])
x+y shape :  torch.Size([3, 3])


Examinons maintenant d'autres exemples plus complexes :


In [14]:
x=torch.zeros(5,3,4,1)
y=torch.zeros(  3,1,1)
print("x+y shape : ",(x+y).shape)
# Le tenseur y a été étendu en taille 1x3x1x1 (règle 1) puis dupliqué en taille 5x3x4x1 (règle 2)

x+y shape :  torch.Size([5, 3, 4, 1])


In [16]:
x=torch.empty(1)
y=torch.empty(3,1,7)
print("x+y shape : ",(x+y).shape)
# Le tenseur y a été étendu en taille 1x1x1 (règle 1) puis dupliqué en taille 3x1x7 (règle 2)

x+y shape :  torch.Size([3, 1, 7])


In [18]:
x=torch.empty(5,2,4,1)
y=torch.empty(3,1,1)
print("x+y shape : ",(x+y).shape)
# L'opération n'est pas possible car les tenseurs ne sont pas broadcastables (dimension 3 en partant de la fin ne correspond pas)

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

## Autres points à considérer


### Comparaison avec des scalaires


On n'y pense pas toujours, mais cela permet d'effectuer des comparaisons de manière simple.


In [22]:
a = torch.tensor([10., 0, -4])
print(a > 0)
print(a==0)

tensor([ True, False, False])
tensor([False,  True, False])


On peut aussi comparer deux tenseurs entre eux :


In [47]:
a=torch.tensor([1,2,3])
b=torch.tensor([4,2,6])
# Comparaison élément par élément
print(a==b)
# Comparaison élément par élément et égalité pour tous les éléments
print((a==b).all())
# Comparaison élément par élément et égalité pour au moins un élément
print((a==b).any())
# Comparaison avec supérieur ou égal
print(a>=b)

tensor([False,  True, False])
tensor(False)
tensor(True)
tensor([False,  True, False])


Cela peut être très utile pour créer des masques à partir d'un seuil, par exemple, ou vérifier que deux opérations sont équivalentes.


### Utilisation de `unsqueeze()`


Nous avons vu précédemment qu'il est possible de *broadcaster* un tenseur de taille $3$ vers une matrice de taille $3 \times 3$. Le *broadcasting* de PyTorch le transforme automatiquement en taille $1 \times 3$ pour effectuer l'opération. Cependant, on peut vouloir réaliser l'opération dans l'autre sens, c'est-à-dire ajouter un tenseur $3 \times 1$ à une matrice de taille $3 \times 3$.

Dans ce cas, il faut remplacer la règle 1 manuellement à l'aide de la fonction [unsqueeze()](https://pytorch.org/docs/stable/generated/torch.unsqueeze.html), qui permet d'ajouter une dimension.


In [29]:
x=torch.tensor([1,2,3])
y=torch.randn(3,3)
print("y : ",y )
print("x+y : ",x+y) 

x=x.unsqueeze(1)
print("x shape : ",x.shape)
print("x+y : ",x+y)

y :  tensor([[ 1.3517,  1.1880,  0.4483],
        [ 0.5137, -0.5406, -0.1412],
        [-0.0108,  1.3757,  0.6112]])
x+y :  tensor([[2.3517, 3.1880, 3.4483],
        [1.5137, 1.4594, 2.8588],
        [0.9892, 3.3757, 3.6112]])
x shape :  torch.Size([3, 1])
x+y :  tensor([[2.3517, 2.1880, 1.4483],
        [2.5137, 1.4594, 1.8588],
        [2.9892, 4.3757, 3.6112]])


Comme vous pouvez le voir, nous avons pu contourner les règles de PyTorch pour obtenir le résultat souhaité.

**Note** :
- La règle 1 de PyTorch équivaut à appliquer *x.unsqueeze(0)* jusqu'à ce que le nombre de dimensions soit le même.
- Il est possible de remplacer *unsqueeze()* par *None* de la manière suivante :


In [32]:
x=torch.tensor([1,2,3])
# La première opération est l'équivalent de unsqueeze(0) et la seconde de unsqueeze(1)
x[None].shape,x[...,None].shape

(torch.Size([1, 3]), torch.Size([3, 1]))

### Utilisation de `keepdim`


Les fonctions de PyTorch qui réduisent la taille d'un tenseur selon une dimension ([*torch.sum*](https://pytorch.org/docs/stable/generated/torch.sum.html) pour sommer selon une dimension, [*torch.mean*](https://pytorch.org/docs/stable/generated/torch.mean.html) pour calculer la moyenne, etc.) ont un paramètre intéressant à utiliser dans certains cas.

Ces opérations modifient la dimension du tenseur et suppriment automatiquement la dimension sur laquelle l'opération a été effectuée.


In [36]:
x=torch.randn(3,4,5)
print(x.shape)
x=x.sum(dim=1) # somme sur la dimension 1
print(x.shape)

torch.Size([3, 4, 5])
torch.Size([3, 5])


Si vous souhaitez conserver la dimension sur laquelle la somme est effectuée, vous pouvez utiliser l'argument *keepdim=True*.


In [37]:
x=torch.randn(3,4,5)
print(x.shape)
x=x.sum(dim=1,keepdim=True) # somme sur la dimension 1
print(x.shape)

torch.Size([3, 4, 5])
torch.Size([3, 1, 5])


Cela peut être très utile pour éviter les erreurs avec les dimensions. Examinons un cas où cela impacte le *broadcasting*.


In [42]:
x=torch.randn(3,4,5)
y=torch.randn(1,1,1)
x_sum=x.sum(dim=1)
x_sum_keepdim=x.sum(dim=1,keepdim=True)
print("Les deux opérations sont elles équivalentes ? :",(x_sum+y==x_sum_keepdim+y).all().item())

Les deux opérations sont elles équivalentes ? : False


Voici ce qui s'est passé :
- Dans le premier cas, *x_sum* a une taille de $3 \times 5$. La règle 1 le transforme en $1 \times 3 \times 5$, et la règle 2 transforme y en $1 \times 3 \times 5$.
- Dans le second cas, *x_sum_keepdim* a une taille de $3 \times 1 \times 5$, et la règle 2 transforme y en $1 \times 3 \times 5$.


### Notation d'Einstein


Cette partie n'est pas directement liée au *broadcasting*, mais il est important de la connaître.

Pour multiplier des matrices dans PyTorch, nous avons utilisé l'opérateur @ (ou *torch.matmul*) jusqu'ici. Il existe une autre méthode pour effectuer des multiplications matricielles avec la *Einstein Summation* ([*torch.einsum*](https://pytorch.org/docs/stable/generated/torch.einsum.html)).

Il s'agit d'une notation compacte pour exprimer des produits et des sommes, par exemple :
**ik,kj -> ij**
Le côté gauche représente les dimensions des entrées, séparées par des virgules. Ici, nous avons deux tenseurs avec chacun deux dimensions (**i,k** et **k,j**). Le côté droit représente les dimensions du résultat, soit un tenseur de dimensions **i,j**.

Les règles de la notation d'Einstein sont :
- Les indices répétés à gauche sont implicitement sommés s'ils n'apparaissent pas à droite.
- Chaque indice peut apparaître au maximum deux fois à gauche.
- Les indices non répétés à gauche doivent apparaître à droite.

On peut l'utiliser pour diverses opérations :
```python
torch.einsum('ij->ji', a)
```
renvoie la transposée de la matrice a.

Alors que
```python
torch.einsum('bi,ij,bj->b', a, b, c)
```
renvoie un vecteur de taille b où la k-ième coordonnée est la somme de $a[k,i]⋅b[i,j]⋅c[k,j]$. Cette notation est particulièrement pratique lorsque vous manipulez des *batchs* avec plusieurs dimensions.
Par exemple, si vous avez deux lots de matrices et que vous voulez calculer le produit matriciel par *batch*, vous pouvez utiliser :
```python
torch.einsum('bik,bkj->bij', a, b)
```


C'est une méthode pratique pour effectuer des multiplications matricielles dans PyTorch. De plus, c'est très rapide et souvent la manière la plus efficace pour réaliser des opérations personnalisées dans PyTorch.
